In [6]:
import torch
import numpy as np
from scipy.special import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json

In [4]:
T = 0.5
threshold = 0.01

model_path = "./notebooks/hate-speech-dynabench-anchored_20x50/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [9]:
def annotate(text):
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Run model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits.cpu().numpy()[0]
    
    # Apply temperature scaling
    probs = softmax(logits / T)[1]    # class 1 probability
    
    # Threshold decision
    pred = int(probs >= threshold)
    
    return {
        "text": text,
        "prob_othering": probs,
        "pred_label": pred,
        "pred_name": "Othering" if pred == 1 else "Not Othering"
    }

In [ ]:
comments = []
with open("C:/Users/hofin/RC_2023-03_results.jsonl", "r") as f:
    for i, line in enumerate(f):
        obj = json.loads(line)
        if obj.get("subreddit") == "changemyview":
            comments.append(obj["body"])
           # or whatever field you need

annotated_comments = [annotate(c) for c in comments]

In [21]:
for item in annotated_comments:
    if(item["pred_name"] == "Othering"):
        print(item["text"])
        print("........")